In [20]:
from tsai.basics import *
import json
import numpy as np
import fastai
import fastai.tabular

from fastai.vision.all import *
from CNN import *
import matplotlib.pyplot as plt
from mocatml.utils import *
from mocatml.data import *

In [ ]:
with open("data_config.json", "r") as outfile:
    data = json.load(outfile, strict=False)
extent = list(data['extent'])
data_numpy = np.load('data.npy')

d, n_epochs, window_size, batch_size = 32, 35, 60, 64
num_sim = 100
model = AutoEncoder(d)
load_model(f'model/d_{d}_epoch_{n_epochs}.pkl', model, None)

In [ ]:
with open(f'model/indices_d_{d}_epoch_{n_epochs}.txt', "r") as f:
    indices = f.read().split()
    
split = float(indices[0])
indices = [int(i) for i in indices[1:]]

train_indices = indices[:int(split * num_sim)]
val_indices = indices[int(split * num_sim):]

In [ ]:
data_enc = []
for i in range(num_sim):
    encoded = model.encrypt(torch.tensor(data_numpy[i], dtype=torch.float32).unsqueeze(0).transpose(0, 1))
    data_enc.append(encoded.detach().numpy())
data_enc = np.array(data_enc)
data_enc.shape

In [ ]:
sh = data_enc.shape
data_enc_reshaped = data_enc.reshape((sh[0], sh[-1], sh[1]))
X_train, y_train = apply_sliding_window_3d(data_enc_reshaped[train_indices], 
                                           window_len=10,
                                           stride=1,
                                           seq_first=False,
                                           horizon=1)
X_valid, y_valid = apply_sliding_window_3d(data_enc_reshaped[val_indices], 
                                           window_len=10,
                                           stride=1,
                                           seq_first=False,
                                           horizon=1)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
X, y, splits = combine_split_data([X_train, X_valid], [y_train, y_valid])
X.shape, y.shape, splits

In [ ]:
torch.backends.cudnn.benchmark = True

In [ ]:
model_name = "PatchTST"
learn = TSForecaster(X, y, splits, bs=batch_size, batch_tfms=[], arch=model_name, arch_config=dict())
lr = learn.lr_find().valley

In [ ]:
epochs = 60
learn.fit_one_cycle(epochs, lr, cbs=[ShowGraphCallback()])